# 📈 Analyse Statistique / Ratings / Long-Terme

**Objectif**: Analyser les tendances à long terme des ratings et détecter les évolutions comportementales.

## 🔍 Questions exploratoires:
- Les utilisateurs deviennent-ils plus/moins sévères dans le temps ?
- Quand la plateforme a-t-elle connu ses pics/creux d'activité ?
- Les power users changent-ils leurs habitudes ?
- Y a-t-il eu des événements perturbateurs ?
- La croissance rapide nuit-elle à la qualité ?

In [1]:
# Import minimal nécessaire pour l'analyse long-term
from pathlib import Path
import sys
import sys
sys.path.append('../../')

# fonctions de chargement de data (internes)
from _data_utils.data_utils_ratings import load_ratings_for_longterm_analysis

# librairies de base
import pandas as pd
import numpy as np

# visualisation minimale utilisée ici (matplotlib suffit)
import matplotlib.pyplot as plt

# statistiques (utilisées pour tests et régression)
from scipy import stats
from scipy.stats import kendalltau, linregress, spearmanr

# Suppression des warnings inutiles
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports minimaux terminés")

# Chargement optimisé des statistiques mensuelles (fonction spécialisée)
print("🔄 Chargement des données avec filtrage robuste optimisé...")
monthly_stats, metadata = load_ratings_for_longterm_analysis(
    min_interactions=100, 
    return_metadata=True, 
    verbose=False
)

print(f"✅ Données long-term chargées: {len(monthly_stats)} mois")
print(f"📊 Mois exclus: {metadata['mois_exclus']} ({metadata['pct_exclus']:.1f}%)")
print(f"📅 Période: {metadata['periode_apres']}")
print(f"📈 Volume moyen: {metadata['volume_moyen_apres']:.0f} interactions/mois")

# Métadonnées disponibles pour analyses ultérieures
print(f"\n💡 Métadonnées disponibles: {list(metadata.keys())}")


✅ _data_utils module chargé (common + ratings + recipes)
✅ Imports minimaux terminés
🔄 Chargement des données avec filtrage robuste optimisé...
✅ Interactions chargées depuis S3 : 681,944 lignes × 6 colonnes
✅ Données long-term chargées: 207 mois
📊 Mois exclus: 17 (7.6%)
📅 Période: 2001-09 → 2018-11
📈 Volume moyen: 3293 interactions/mois

💡 Métadonnées disponibles: ['seuil_applique', 'mois_total', 'mois_exclus', 'mois_conserves', 'pct_exclus', 'periode_avant', 'periode_apres', 'gaps_temporels', 'volume_moyen_avant', 'volume_moyen_apres']


## 📊 Analyse 1: Tendance temporelle des ratings

Analyse de l'évolution des ratings dans le temps pour détecter les tendances monotones et quantifier les changements comportementaux.

### Visualisation

In [ ]:
# Visualisation complète des tendances temporelles
print("📊 Création des visualisations temporelles...")

# Préparation des données pour la visualisation
X = np.arange(len(monthly_stats))

# Calcul de la régression linéaire pour la ligne de tendance
from scipy.stats import linregress
trend_slope, intercept, r_value, p_value_reg, std_err = linregress(X, monthly_stats['mean_rating'].values)

# Visualisation en 4 graphiques
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Tendance des ratings avec ligne de tendance
ax1.plot(monthly_stats['date'], monthly_stats['mean_rating'], 'bo-', linewidth=2, markersize=4, label='Rating moyen')
trend_line = intercept + trend_slope * X
ax1.plot(monthly_stats['date'], trend_line, 'r--', linewidth=2, label=f'Tendance ({trend_slope:.4f}/mois)')
ax1.set_title("Évolution des ratings temporelle", fontsize=14, fontweight='bold')
ax1.set_ylabel("Rating moyen")
ax1.grid(True, alpha=0.3)
ax1.legend()

# 2. Volume d'interactions
ax2.plot(monthly_stats['date'], monthly_stats['n_interactions'], 'go-', linewidth=2, markersize=4)
ax2.set_title("Volume d'interactions", fontsize=14, fontweight='bold')
ax2.set_ylabel("Nombre d'interactions")
ax2.grid(True, alpha=0.3)

# 3. Stabilité (écart-type)
ax3.plot(monthly_stats['date'], monthly_stats['std_rating'], 'o-', color='orange', linewidth=2, markersize=4)
ax3.set_title("Stabilité (écart-type)", fontsize=14, fontweight='bold')
ax3.set_ylabel("Écart-type des ratings")
ax3.set_xlabel("Date")
ax3.grid(True, alpha=0.3)

# 4. Scatter volume vs qualité
ax4.scatter(monthly_stats['n_interactions'], monthly_stats['mean_rating'], 
           color='purple', alpha=0.7, s=50)
ax4.set_title("Corrélation volume-qualité", fontsize=14, fontweight='bold')
ax4.set_xlabel("Nombre d'interactions")
ax4.set_ylabel("Rating moyen")
ax4.grid(True, alpha=0.3)

# Ligne de corrélation sur le scatter
z = np.polyfit(monthly_stats['n_interactions'], monthly_stats['mean_rating'], 1)
p = np.poly1d(z)
ax4.plot(monthly_stats['n_interactions'], p(monthly_stats['n_interactions']), 
         "r--", alpha=0.8, linewidth=2)

plt.suptitle(f"Analyse Temporelle - Pente: {trend_slope:+.4f}/mois", 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ Visualisations créées")

### Tests statistiques

In [ ]:
# Tests statistiques pour détecter les tendances temporelles
print("=" * 60)
print("📈 TESTS STATISTIQUES - TENDANCE TEMPORELLE")
print("=" * 60)

# Test de Mann-Kendall pour détecter tendance monotone
def mann_kendall_test(data):
    """Implémentation du test de Mann-Kendall pour détecter les tendances"""
    n = len(data)
    s = 0
    
    for i in range(n-1):
        for j in range(i+1, n):
            if data[j] > data[i]:
                s += 1
            elif data[j] < data[i]:
                s -= 1
    
    # Variance
    var_s = n * (n-1) * (2*n+5) / 18
    
    # Statistique de test
    if s > 0:
        z = (s - 1) / np.sqrt(var_s)
    elif s < 0:
        z = (s + 1) / np.sqrt(var_s)
    else:
        z = 0
    
    # P-value (bilatéral)
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    return z, p_value, s

# Application du test de Mann-Kendall
z_mk, p_mk, s_mk = mann_kendall_test(monthly_stats['mean_rating'].values)

print(f"🔍 Test de Mann-Kendall:")
print(f"   Statistique S: {s_mk}")
print(f"   Z-score: {z_mk:.4f}")
print(f"   P-value: {p_mk:.4f}")

interpretation = "TENDANCE SIGNIFICATIVE" if p_mk < 0.05 else "PAS DE TENDANCE CLAIRE"
direction = "HAUSSE" if s_mk > 0 else "BAISSE" if s_mk < 0 else "STABLE"
print(f"   🎯 Résultat: {interpretation} - Direction: {direction}")

# Régression linéaire pour quantifier la tendance
X = np.arange(len(monthly_stats))
y = monthly_stats['mean_rating'].values
trend_slope, intercept, r_value, p_value_reg, std_err = linregress(X, y)
trend_r2 = r_value ** 2

print(f"\n📊 Régression linéaire:")
print(f"   Pente: {trend_slope:.6f} points/mois")
print(f"   R²: {trend_r2:.4f}")
print(f"   P-value: {p_value_reg:.4f}")
print(f"   Évolution annuelle: {trend_slope * 12:.4f} points/an")

# Corrélation volume vs qualité
from scipy.stats import spearmanr
corr_vol_qual, p_corr = spearmanr(monthly_stats['n_interactions'], monthly_stats['mean_rating'])
print(f"\n🔗 Corrélation Volume-Qualité:")
print(f"   ρ de Spearman: {corr_vol_qual:.3f}")
print(f"   P-value: {p_corr:.4f}")

print("\n" + "=" * 60)

### Interprétation

> **💡 Interprétation statistique:**
>
> Les **tests de Mann-Kendall** et la **régression linéaire** révèlent la présence (ou l'absence) d'une **tendance temporelle significative** dans l'évolution des ratings. 
>
> La **pente de régression** quantifie l'ampleur du changement mensuel, tandis que le **R²** indique la fiabilité de cette tendance. La **corrélation volume-qualité** permet d'identifier si l'activité de la plateforme influence la sévérité des utilisateurs.

> **💡 Interprétation opérationnelle:**
>
> Ces résultats permettent de **comprendre l'évolution comportementale** des utilisateurs dans le temps. Une **tendance négative** suggérerait que les utilisateurs deviennent plus sévères, tandis qu'une **tendance positive** indiquerait une indulgence croissante.
>
> La **corrélation volume-qualité** révèle si les **pics d'activité** coïncident avec une **dégradation** ou **amélioration** de la qualité perçue, informant sur l'impact de la croissance sur l'expérience utilisateur.

## 📊 Analyse 2: Évolution détaillée et corrélations

Analyse approfondie de l'évolution des ratings avec bandes de confiance et corrélations volume-qualité pour identifier les patterns comportementaux.

### Visualisation

In [ ]:
# Préparation des données pour l'analyse détaillée
import matplotlib.dates as mdates

# Préparation robuste du DataFrame
monthly_df = monthly_stats.copy()
monthly_df['date'] = pd.to_datetime(monthly_df['date'])
monthly_df = monthly_df.sort_values('date')
monthly_df['mean_rating'] = pd.to_numeric(monthly_df['mean_rating'], errors='coerce')
monthly_df['std_rating'] = pd.to_numeric(monthly_df.get('std_rating', 0), errors='coerce').fillna(0)
monthly_df['n_interactions'] = pd.to_numeric(monthly_df['n_interactions'], errors='coerce').fillna(0)
monthly_df = monthly_df.dropna(subset=['mean_rating'])

# 1. Évolution avec bandes de confiance
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))

# Graphique 1: Rating avec bandes de confiance
ax1.plot(monthly_df['date'], monthly_df['mean_rating'], 'o-', 
        color='#1f77b4', linewidth=2, markersize=4, label='Rating moyen')

# Bandes de confiance (±1 std)
ax1.fill_between(monthly_df['date'].values, 
                (monthly_df['mean_rating'] - monthly_df['std_rating']).values,
                (monthly_df['mean_rating'] + monthly_df['std_rating']).values,
                alpha=0.2, color='#1f77b4', label='±1 écart-type')

ax1.set_title('Évolution des ratings avec bandes de confiance', fontsize=16, fontweight='bold')
ax1.set_ylabel('Rating moyen', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend()

# Graphique 2: Volume des interactions
ax2.plot(monthly_df['date'], monthly_df['n_interactions'], color='green', linewidth=2, label='Volume mensuel')
ax2.set_title('Volume mensuel d\'interactions', fontsize=16, fontweight='bold')
ax2.set_ylabel('Nombre d\'interactions', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend()

# Formatage des dates
for ax in [ax1, ax2]:
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

plt.tight_layout()
plt.show()

print("✅ Visualisations détaillées créées")

### Tests statistiques

In [ ]:
# Tests statistiques avancés pour corrélations et tendances
print("=" * 60)
print("📈 TESTS STATISTIQUES - CORRÉLATIONS AVANCÉES")
print("=" * 60)

# Préparation des données pour tests
monthly_sorted = monthly_df.sort_values('date')
time_index = range(len(monthly_sorted))
ratings = monthly_sorted['mean_rating'].values
volumes = monthly_sorted['n_interactions'].values

# Test de Kendall pour tendance temporelle
from scipy.stats import kendalltau
tau, p_value_kendall = kendalltau(time_index, ratings)

print(f"🔍 Test de tendance temporelle (Kendall):")
print(f"   Tau de Kendall: {tau:.4f}")
print(f"   P-value: {p_value_kendall:.4f}")

if p_value_kendall < 0.05:
    direction = "croissante" if tau > 0 else "décroissante"
    print(f"   ✅ Tendance {direction} significative")
else:
    print(f"   ❌ Pas de tendance temporelle significative")

# Corrélation volume-rating
correlation = monthly_df[['n_interactions', 'mean_rating']].corr().iloc[0, 1] if len(monthly_df) > 1 else float('nan')
corr_spearman, p_corr_spearman = spearmanr(volumes, ratings)

print(f"\n🔗 Corrélation Volume-Rating:")
print(f"   Corrélation de Pearson: {correlation:.3f}")
print(f"   Corrélation de Spearman: {corr_spearman:.3f}")
print(f"   P-value (Spearman): {p_corr_spearman:.4f}")

# Interprétation automatique de la corrélation
if not pd.isna(correlation):
    if p_corr_spearman < 0.05:
        if abs(corr_spearman) > 0.3:
            print(f"   ⚠️ Corrélation {'positive' if corr_spearman > 0 else 'négative'} notable et significative")
        else:
            print(f"   ✅ Corrélation {'positive' if corr_spearman > 0 else 'négative'} faible mais significative")
    else:
        print(f"   ℹ️ Pas de corrélation significative volume-qualité")
else:
    print(f"   ⚠️ Données insuffisantes pour calculer la corrélation")

# Régression linéaire détaillée
slope, intercept, r_value, p_value_reg, std_err = linregress(time_index, ratings)
annual_slope = slope * 12

print(f"\n📊 Régression temporelle détaillée:")
print(f"   Pente: {slope:.6f} points/mois")
print(f"   R²: {r_value**2:.4f}")
print(f"   P-value: {p_value_reg:.4f}")
print(f"   Erreur standard: {std_err:.6f}")
print(f"   📅 Évolution annuelle estimée: {annual_slope:.4f} points/an")

print("\n" + "=" * 60)

### Interprétation

> **💡 Interprétation statistique:**
>
> L'analyse des **corrélations volume-qualité** et des **tendances temporelles** avec les tests de **Kendall** et la **régression linéaire** révèle les **dynamiques comportementales** de la plateforme.
>
> Les **bandes de confiance** (±1 écart-type) permettent d'identifier les **périodes de volatilité** versus les **phases de stabilité** dans les ratings, informant sur la **cohérence** des évaluations utilisateurs.

> **💡 Interprétation opérationnelle:**
>
> Ces résultats permettent d'identifier si les **pics d'activité** de la plateforme coïncident avec une **amélioration** ou **dégradation** de la satisfaction utilisateur.
>
> Une **corrélation positive volume-qualité** suggère que l'**engagement** améliore l'expérience, tandis qu'une **corrélation négative** pourrait indiquer des problèmes de **scalabilité** ou d'**inflation** des attentes lors des pics d'usage.